In [1]:
from box import ConfigBox

In [2]:
d = {"key":"value","key1":"value1"}

In [4]:
d = ConfigBox(d)

In [5]:
d.key

'value'

### Downlaod Data from google drive

In [6]:
import gdown

In [7]:
url = "https://drive.google.com/file/d/1gvXLFxxoqawVubViRigjSK0m8VPsJHWM/view?usp=sharing"

In [8]:
type(url)

str

In [12]:
fileId = url.split("/")[-2]
fileId

'1gvXLFxxoqawVubViRigjSK0m8VPsJHWM'

In [13]:
prefix = 'https://drive.google.com/uc?/export=download&id='
gdown.download(prefix+fileId, "SentimentDataFinal.zip")

Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=1gvXLFxxoqawVubViRigjSK0m8VPsJHWM
From (redirected): https://drive.google.com/uc?/export=download&id=1gvXLFxxoqawVubViRigjSK0m8VPsJHWM&confirm=t&uuid=105f37c5-b9e6-483c-b0aa-390a3643b5ba
To: c:\Users\akish\SentimentAnalysis\SentimentAPI\research\SentimentDataFinal.zip
100%|██████████| 74.5M/74.5M [00:27<00:00, 2.67MB/s]


'SentimentDataFinal.zip'

In [14]:
import os

In [3]:
%pwd

'c:\\Users\\akish.pothuri\\python\\SentimentAnalysis\\SentimentAPI\\research'

In [4]:
import os
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\akish.pothuri\\python\\SentimentAnalysis\\SentimentAPI'

In [18]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [19]:
from Sentiment.constants import *
from Sentiment.utils.common import read_yaml, create_directories

In [20]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast

In [7]:
data = pd.read_excel('artifacts\data_transformation\sentimentDataset\TransformedData.xlsx')

In [7]:
%pip install torch

In [23]:
%pip install transformers==4.6.0

     ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
     -------- ------------------------------- 0.5/2.3 MB 15.2 MB/s eta 0:00:01
     --------------- ------------------------ 0.9/2.3 MB 11.2 MB/s eta 0:00:01
     -------------------- ------------------- 1.2/2.3 MB 11.0 MB/s eta 0:00:01
     -------------------------- ------------- 1.5/2.3 MB 9.7 MB/s eta 0:00:01
     ----------------------------- ---------- 1.7/2.3 MB 7.9 MB/s eta 0:00:01
     ----------------------------------- ---- 2.1/2.3 MB 8.3 MB/s eta 0:00:01
     ---------------------------------------  2.3/2.3 MB 7.8 MB/s eta 0:00:01
     ---------------------------------------- 2.3/2.3 MB 7.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
     ------- -------------------------------- 0.4/2.0 MB 7.4 MB/s eta 0:00:01
     -------------- ------------------------- 0.7/2.0 MB 9.3 MB/s eta 0:00:01
     ---------------------- ----------------- 1.1/2.0 MB 7.9 MB/s eta 0:00:

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.16.1 requires huggingface-hub>=0.19.4, but you have huggingface-hub 0.0.8 which is incompatible.

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
data.drop('Unnamed: 0',axis=1,inplace=True)
data.dropna(how="any",inplace=True)

In [9]:
Review=list(data['Review'])
sentiment=list(data['sentiment'])

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Review, sentiment, test_size = 0.20, random_state = 0)

In [24]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [25]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)

In [26]:
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [27]:
import tensorflow as tf 

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),y_test
))

In [28]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./results/logs',
    logging_steps=10
)

In [29]:
data.isnull().sum()

Review       0
sentiment    0
dtype: int64

In [30]:
with training_args.strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = TFTrainer(
    model = model,
    args = training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [31]:
trainer.train()

TypeError: '>' not supported between instances of 'NoneType' and 'int'